In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import nltk
import json
import os
import sys
import time

In [12]:
########### Set up Output File
outdir = '..//data//clean_data'

##### Contain 2 column : 1. Replaced name of entity / 2. Real name of entity
reverse_entity = open(os.path.join(outdir,os.path.basename('entity_total.csv')),'w') 

#####
entity_replaced = open(os.path.join(outdir,os.path.basename('entity_replaced.csv')),'w',encoding='utf-8')



In [13]:
########### Trigger word
triggers_raw = open('..//data//preprocessing//triggers.txt').read().split('\n')[:-1]
triggers = []
for row in triggers_raw :
    triggers.append(row.split('\t')[-1])

In [14]:
########### Entity sources

#### Drug synonyms from drugbank
drugs = json.load(open('..//data/preprocessing/drug_corpus/drug_name_synonyms.json'))

#### Herb scientific name and its synonyms (not alias)
herbs = json.load(open('..//data/preprocessing/herb_corpus/herb_science_synonyms.json'))

#### standard ref without effect
ref_without = json.load(open('..//data/preprocessing/standard_ref/standard_ref_without_effect.json'))

#### standard ref with effect
ref_effect = open('..//data/preprocessing/standard_ref/standard_ref_effect.csv').read().split('\n')[:-1]

#### Adverse effect corpus
adverse_effect = json.load(open('..//data/preprocessing/adverse_effect_corpus/adverse_effect.json'))





In [15]:
########## Create name for entity

###### trigger entity
trigger_entity = {}
for each in triggers :
    trigger_entity[each] = 'trigger_entity'
    reverse_entity.write('trigger_entity\t%s\n'%each)

###### Drug entity
drug_entity = {}
drugorder = 0
for ID in drugs :
    for drug in drugs[ID] :
        if drug.lower() in herbs or drug.lower() in ref_without :
            break
        drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
        try :
            reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drug.lower()))
        except :
            drugencode = str(drug.lower()).encode('utf8')
            reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drugencode))
    drugorder+=1

###### Herb entity 
herb_entity = {}
herborder = 0
for herb in herbs :
    herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
    reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herb.lower()))
    for synonym in herbs[herb] :
        herb_entity[synonym.lower()] = 'herb_entity_%s'%herborder
        try :
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,synonym.lower()))
        except :
            herbencode = str(synonym.lower()).encode('utf8')
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herbencode))
    herborder+=1
    
###### Adverse effect entity
effect_entity = {}
effectorder = 0
for each in adverse_effect :
    effect_entity[each.lower()] = 'EF_entity_%s'%effectorder
    try :
        reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,each.lower()))
    except :
        effectencode = str(each.lower()).encode('utf8')
        reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
    effectorder+=1
    
###### standard ref without effect entity 
for herb in ref_without :
    if herb.lower() not in herb_entity :
        herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
        try :
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herb.lower()))
        except :
            herbencode = str(herb.lower()).encode('utf8')
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herbencode))
        herborder+=1
    for drug in ref_without[herb] :
        if drug.lower() not in drug_entity :
            drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
            try :
                reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drug.lower()))
            except :
                drugencode = str(drug.lower()).encode('utf8')
                reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drugencode))
            drugorder+=1
            
###### standard ref with effect
for row in ref_effect :
    effect = row.split('\t')[2]
    ###
    if effect.lower() not in effect_entity :
        effect_entity[effect.lower()] = 'EF_entity_%s'%effectorder
        try :
            reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effect.lower()))
        except :
            effectencode = str(effect.lower()).encode('utf8')
            reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
        effectorder+=1
    ###    
    if len(row.split('\t')) == 4 :
        effect1 = row.split('\t')[3]
        if effect1.lower() not in effect_entity :
            effect_entity[effect1.lower()] = 'EF_entity_%s'%effectorder
            try :
                reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effect1.lower()))
            except :
                effectencode = str(effect1.lower()).encode('utf8')
                reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
            effectorder+=1
######
reverse_entity.close()

In [16]:
###### remove '\n' from text and merge text
def rmv_n(textdict) :
    text = ''
    for each in textdict :
        if len(textdict[each].split('\n')) != 1 :
            new = ''.join(textdict[each].split('\n'))
            text+=new
        else:
            text+= textdict[each]

    text = text.lower()
    return text

In [17]:
###### Entity replacement

def entity_replace(text) :
    ###
    begin = time.time()
    ###
    herb_entity_single = {}
    for herb in herb_entity :
        if herb in text :
            text = text.replace(herb,' %s '%herb_entity[herb])
            entity_replaced.write('%s\t%s\n'%(herb_entity[herb],herb))
    print('herb done')
    ###
    drug_entity_single = {}
    for drug in drug_entity :
        if len(drug) > 4 :
            if drug in text :
                text = text.replace(drug,' %s '%drug_entity[drug])
                entity_replaced.write('%s\t%s\n'%(drug_entity[drug],drug))
        else:
            drug_entity_single[drug] = drug_entity[drug]
    print('drug done')
    ###
    for effect in effect_entity :
        if len(effect) > 3 :
            if effect in text :
                text = text.replace(effect,' %s '%effect_entity[effect])
                entity_replaced.write('%s\t%s\n'%(effect_entity[effect],effect))
    print('effect done')
    ###
    for trig in trigger_entity:
        if trig in text:
            text = text.replace(trig,' %s '%trigger_entity[trig])
            entity_replaced.write('%s\t%s\n'%(trigger_entity[trig],trig))
    print('trigger done')
    print(time.time() - begin)
    print('replace entities done')
    return text

In [18]:
######## word processing and word removal
def word_processing(text) :
    begin = time.time()
    count = 0

    ###### Create pos_tag accept list and stop words list 
    stop_words = list(set(stopwords.words('english')))
    pos_tag_accept = ['NN','NNS','NNP','NNPS','VB','VBD','VBG','VBN','VBP','VBZ']

    ###### Create entity list 
    reverse_entity = open('..//data/clean_data/entity_total.csv').read().split('\n')[:-1]
    entity_list = []
    for row in reverse_entity :
        entity_list.append(row.split('\t')[0])

    entity_list = list(set(entity_list))

    ###### Split text into sentences
    sentences = TextBlob(text).sentences
    total = len(sentences)
    print('Processing %s sentences'%total)
    sentences_processed = []

    ###### Processing
    for sentence in sentences :          
        words = word_tokenize(str(sentence)) ##### Split sentence into words and normalization
        #####
        if len(set(words).intersection(entity_list)) > 2 : ##### Choose sentence contain more than 2 entities
            for word in words :
                if word in stop_words : ##### Remove stop words
                    words.pop(words.index(word))
                elif len(word) == 1 : ##### Remove punctuation marks
                    words.pop(words.index(word))    
            ###
            tags = nltk.pos_tag(words)
            for tag in tags :
                if tag[0] in entity_list : ##### Save entities
                    pass      
                elif tag[1] not in pos_tag_accept : ##### Remove words which are not noun,verb, and adjective 
                    words.pop(words.index(tag[0]))
            ###
            if len(words) > 3 : ###### Save processed sentence
                sentences_processed.append(words)
        ###
        if count % 10000 == 0 :
            print('Processed %s sentences'%count)
        count+=1

    ##### 
    print(time.time() - begin)
    print('Word processed done')
    return sentences_processed


In [19]:
########### Text source
text_list = os.listdir('..//data/preprocessing/text_extract/')
#text_list = ['text_extract.json']
count = 0
for file in text_list :
    print('Process - %s'%file)
    textdict = json.load(open('..//data//preprocessing//text_extract//%s'%file))
    #textdict = json.load(open('..//data//preprocessing//%s'%file))
    #####
    print('step 1')
    text = rmv_n(textdict)
    textdict = []
    #####
    print('step 2')
    text = entity_replace(text)
    #####
    print('step 3')
    sentences_processed = word_processing(text)
    #####
    ##### List of processed sentences
    outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%count)),'w')
    outfile.write(json.dumps(sentences_processed))
    outfile.close()
    count+=1
#####
entity_replaced.close()

Process - text_extract_0.json
step 1
step 2
herb done
drug done
effect done
trigger done
1393.9080746173859
replace entities done
step 3
Processing 89849 sentences
Processed 0 sentences
Processed 10000 sentences
Processed 20000 sentences
Processed 30000 sentences
Processed 40000 sentences
Processed 50000 sentences
Processed 60000 sentences
Processed 70000 sentences
Processed 80000 sentences
1506.349447965622
Word processed done
Process - text_extract_1.json
step 1
step 2
herb done
drug done
effect done
trigger done
1076.7565824985504
replace entities done
step 3
Processing 93155 sentences
Processed 0 sentences
Processed 10000 sentences
Processed 20000 sentences
Processed 30000 sentences
Processed 40000 sentences
Processed 50000 sentences
Processed 60000 sentences
Processed 70000 sentences
Processed 80000 sentences
Processed 90000 sentences
1634.3284029960632
Word processed done
Process - text_extract_10.json
step 1
step 2
herb done
drug done
effect done
trigger done
1477.8452010154724
